# Pipeline de Previsão com Visualizações Completas

Este notebook demonstra todas as visualizações disponíveis no sistema.

In [ ]:
import sys
sys.path.append('../src')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from pipeline import VitoriaForecastPipeline
from utils.visualization import VitoriaVisualizer

# Configurações
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')
%matplotlib inline

print("✓ Bibliotecas carregadas")

## 1. Carregamento de Dados

In [ ]:
# EXEMPLO COM DADOS SINTÉTICOS
# Substitua por: df = pd.read_csv('seus_dados.csv', index_col=0, parse_dates=True)

np.random.seed(42)
dates = pd.date_range('2010-01-01', periods=120, freq='MS')

# Simula séries correlacionadas
trend = np.linspace(0, 2, 120)
cycle = 0.5 * np.sin(2 * np.pi * np.arange(120) / 12)

df = pd.DataFrame({
    'preco_m2': trend + cycle + 0.2 * np.random.randn(120).cumsum(),
    'lancamentos': 0.8 * trend + 0.3 * cycle + 0.15 * np.random.randn(120).cumsum(),
    'credito_imob': 0.9 * trend + 0.4 * cycle + 0.1 * np.random.randn(120).cumsum(),
    'emprego_construcao': 0.7 * trend + 0.2 * cycle + 0.2 * np.random.randn(120).cumsum(),
    'massa_salarial': 0.6 * trend + 0.1 * cycle + 0.15 * np.random.randn(120).cumsum(),
    'pib_es': 0.85 * trend + 0.25 * cycle + 0.1 * np.random.randn(120).cumsum(),
    'selic': -0.3 * trend + 0.1 * np.random.randn(120).cumsum(),
}, index=dates)

print(f"Dados: {df.shape[0]} observações, {df.shape[1]} variáveis")
print(f"Período: {df.index[0].strftime('%Y-%m')} a {df.index[-1].strftime('%Y-%m')}")

## 2. Execução do Pipeline

In [ ]:
# Inicializa pipeline
pipeline = VitoriaForecastPipeline(
    max_vars=5,
    forecast_horizon=12,
    ar_order=2,
    verbose=True
)

# Executa
results = pipeline.run_full_pipeline(
    df,
    models_to_train=['arima', 'ridge', 'lasso', 'random_forest', 'quantile'],
    ensemble_method='weighted_avg'
)

## 3. Visualizações

### 3.1. IDCI-VIX Histórico

In [ ]:
# Inicializa visualizador
viz = VitoriaVisualizer(figsize=(14, 7))

# Plota IDCI-VIX
idci_vix = results['idci_vix']

fig1 = viz.plot_idci_vix(
    idci_vix,
    show_zones=True,
    save_path='../data/processed/idci_vix_historico.png'
)
plt.show()

print(f"\n📊 IDCI-VIX atual: {idci_vix.iloc[-1]:.2f}")
print(f"   Média histórica: {idci_vix.mean():.2f}")
print(f"   Desvio padrão: {idci_vix.std():.2f}")

### 3.2. Comparação de Modelos

In [ ]:
# Compara todas as previsões
forecasts = results['forecasts']

fig2 = viz.plot_forecasts_comparison(
    historical=idci_vix,
    forecasts_dict=forecasts,
    save_path='../data/processed/comparacao_modelos.png'
)
plt.show()

### 3.3. Previsão com Intervalos de Confiança

In [ ]:
# Usa regressão quantílica para intervalos
if 'quantile_quantiles' in forecasts:
    quantiles = forecasts['quantile_quantiles']
    
    # Prepara dados
    last_date = idci_vix.index[-1]
    freq = pd.infer_freq(idci_vix.index) or 'MS'
    future_dates = pd.date_range(start=last_date, periods=13, freq=freq)[1:]
    
    forecast_median = pd.Series(quantiles['q0.5'].values[:12], index=future_dates)
    forecast_lower = pd.Series(quantiles['q0.1'].values[:12], index=future_dates)
    forecast_upper = pd.Series(quantiles['q0.9'].values[:12], index=future_dates)
    
    fig3 = viz.plot_forecast_with_intervals(
        historical=idci_vix,
        forecast_median=forecast_median,
        forecast_lower=forecast_lower,
        forecast_upper=forecast_upper,
        save_path='../data/processed/previsao_intervalos.png'
    )
    plt.show()
else:
    print("⚠ Regressão quantílica não disponível")

### 3.4. Análise de Cenários

In [ ]:
if 'quantile_quantiles' in forecasts:
    scenarios_df = forecasts['quantile_quantiles'].iloc[:12]
    
    fig4 = viz.plot_scenarios(
        historical=idci_vix,
        scenarios_df=scenarios_df,
        save_path='../data/processed/cenarios.png'
    )
    plt.show()
    
    # Tabela de cenários
    print("\n" + "="*70)
    print("CENÁRIOS DE PREVISÃO (12 MESES)")
    print("="*70)
    
    scenarios_table = pd.DataFrame({
        'Horizonte': range(1, 13),
        'Pessimista (10%)': scenarios_df['q0.1'].values[:12],
        'Base (50%)': scenarios_df['q0.5'].values[:12],
        'Otimista (90%)': scenarios_df['q0.9'].values[:12]
    })
    
    print(scenarios_table.to_string(index=False, float_format='%.2f'))
    
    # Interpretação
    current = idci_vix.iloc[-1]
    forecast_12m = scenarios_df['q0.5'].iloc[-1]
    
    print("\n" + "="*70)
    print("INTERPRETAÇÃO")
    print("="*70)
    print(f"\nIDCI-VIX atual: {current:.2f}")
    print(f"Previsão 12 meses (base): {forecast_12m:.2f}")
    print(f"Variação: {((forecast_12m/current - 1) * 100):.1f}%")
    
    if forecast_12m > 7:
        trend = "🔴 AQUECIMENTO FORTE"
    elif forecast_12m > 5:
        trend = "🟠 AQUECIMENTO MODERADO"
    elif forecast_12m > 3:
        trend = "🟡 ESTABILIDADE"
    else:
        trend = "🔵 DESACELERAÇÃO"
    
    print(f"\nTendência: {trend}")

### 3.5. Feature Importance (Random Forest)

In [ ]:
# Feature importance do Random Forest
if 'random_forest' in results['models']:
    rf_model = results['models']['random_forest']
    
    # Pega importância do primeiro horizonte
    if hasattr(rf_model, 'models') and 1 in rf_model.models:
        model_h1 = rf_model.models[1]['model']
        feature_names = rf_model.models[1]['feature_names']
        
        importance_df = pd.DataFrame({
            'feature': feature_names,
            'importance': model_h1.feature_importances_
        }).sort_values('importance', ascending=False)
        
        fig5 = viz.plot_feature_importance(
            importance_df,
            top_k=15,
            model_name='Random Forest (h=1)',
            save_path='../data/processed/feature_importance.png'
        )
        plt.show()
        
        print("\n📊 Top-10 Features mais importantes:")
        print(importance_df.head(10).to_string(index=False, float_format='%.4f'))
    else:
        print("⚠ Modelo Random Forest não tem feature importance disponível")
else:
    print("⚠ Random Forest não foi treinado")

### 3.6. Análise de Regimes (se Markov disponível)

In [ ]:
# Análise de regimes (Markov-switching)
if 'markov' in results['models']:
    markov_model = results['models']['markov']
    
    try:
        # Pega probabilidades de regime
        regime_probs = markov_model.get_regime_probabilities(smoothed=True)
        
        fig6 = viz.plot_regimes(
            data=idci_vix,
            regime_probs=regime_probs,
            threshold=0.7,
            save_path='../data/processed/regimes.png'
        )
        plt.show()
        
        # Estatísticas por regime
        print("\n" + "="*70)
        print("ANÁLISE DE REGIMES")
        print("="*70)
        
        for regime in [0, 1]:
            col = f'regime_{regime}'
            prob_mean = regime_probs[col].mean()
            in_regime = (regime_probs[col] > 0.7).sum()
            
            regime_name = "BAIXO (Contração)" if regime == 0 else "ALTO (Expansão)"
            
            print(f"\nRegime {regime} - {regime_name}:")
            print(f"  Probabilidade média: {prob_mean:.1%}")
            print(f"  Períodos neste regime: {in_regime} ({in_regime/len(regime_probs)*100:.1f}%)")
        
        # Regime atual
        current_regime = regime_probs.iloc[-1].idxmax()
        current_prob = regime_probs.iloc[-1].max()
        
        print(f"\n🎯 Regime atual: {current_regime} (prob: {current_prob:.1%})")
        
    except Exception as e:
        print(f"⚠ Erro ao analisar regimes: {e}")
else:
    print("⚠ Modelo Markov-switching não foi treinado")

### 3.7. Validação: Treino vs Previsão (Exemplo com ARIMA)

In [ ]:
# Exemplo de validação com ARIMA
if 'arima' in results['models']:
    arima_model = results['models']['arima']
    
    try:
        # Pega fitted values (treino)
        train_predictions = arima_model.get_insample_predictions()
        train_data = idci_vix.loc[train_predictions.index]
        
        # Previsão (teste simulado - últimos 12 meses)
        split_idx = len(idci_vix) - 12
        test_data = idci_vix.iloc[split_idx:]
        
        # Faz previsão out-of-sample (simplificado)
        test_predictions = forecasts['arima']['forecast'].iloc[:len(test_data)]
        test_predictions.index = test_data.index
        
        fig7 = viz.plot_training_vs_prediction(
            train_data=train_data,
            train_predictions=train_predictions,
            test_data=test_data,
            test_predictions=test_predictions,
            model_name='ARIMA',
            save_path='../data/processed/arima_train_test.png'
        )
        plt.show()
        
        # Métricas
        from sklearn.metrics import mean_squared_error, mean_absolute_error
        
        train_rmse = np.sqrt(mean_squared_error(train_data, train_predictions))
        test_rmse = np.sqrt(mean_squared_error(test_data, test_predictions))
        
        print("\n📊 Métricas ARIMA:")
        print(f"  RMSE (treino): {train_rmse:.4f}")
        print(f"  RMSE (teste):  {test_rmse:.4f}")
        
    except Exception as e:
        print(f"⚠ Erro ao validar ARIMA: {e}")
else:
    print("⚠ ARIMA não foi treinado")

## 4. Exportação de Resultados

In [ ]:
# Salva resultados principais
idci_vix.to_csv('../data/processed/idci_vix.csv', header=True)
results['ensemble'].to_csv('../data/processed/forecast_ensemble_12m.csv')

if 'quantile_quantiles' in forecasts:
    forecasts['quantile_quantiles'].to_csv('../data/processed/forecast_scenarios_12m.csv')

print("\n✅ Resultados salvos em ../data/processed/")
print("\n📁 Arquivos gerados:")
print("  - idci_vix.csv")
print("  - forecast_ensemble_12m.csv")
print("  - forecast_scenarios_12m.csv")
print("  - Gráficos PNG (vários)")

## 5. Resumo Executivo

In [ ]:
print("\n" + "#"*80)
print("# RESUMO EXECUTIVO - MERCADO IMOBILIÁRIO VITÓRIA/ES")
print("#"*80)

print(f"\n📊 SITUAÇÃO ATUAL")
print(f"  IDCI-VIX: {idci_vix.iloc[-1]:.2f}")
print(f"  Data: {idci_vix.index[-1].strftime('%Y-%m')}")

if idci_vix.iloc[-1] > 7:
    status = "🔴 AQUECIMENTO FORTE"
elif idci_vix.iloc[-1] > 5:
    status = "🟠 AQUECIMENTO MODERADO"
elif idci_vix.iloc[-1] > 3:
    status = "🟡 ESTÁVEL"
else:
    status = "🔵 RESFRIADO"

print(f"  Status: {status}")

print(f"\n🔮 PREVISÃO 12 MESES")
ensemble_12m = results['ensemble']['forecast'].iloc[0]
print(f"  Ensemble: {ensemble_12m:.2f}")

if 'quantile_quantiles' in forecasts:
    q10 = forecasts['quantile_quantiles']['q0.1'].iloc[-1]
    q50 = forecasts['quantile_quantiles']['q0.5'].iloc[-1]
    q90 = forecasts['quantile_quantiles']['q0.9'].iloc[-1]
    
    print(f"  Pessimista (10%): {q10:.2f}")
    print(f"  Base (50%): {q50:.2f}")
    print(f"  Otimista (90%): {q90:.2f}")

print(f"\n📈 VARIÁVEIS MAIS IMPORTANTES")
for i, var in enumerate(results['selected_vars'], 1):
    print(f"  {i}. {var}")

print(f"\n🤖 MODELOS TREINADOS")
for model in results['models'].keys():
    print(f"  ✓ {model.upper()}")

print("\n" + "#"*80)

## Conclusão

Este notebook fornece:

1. ✅ **IDCI-VIX** - Índice sintético (0-10)
2. ✅ **Previsões 12M** - Ensemble de múltiplos modelos
3. ✅ **Cenários** - Pessimista/Base/Otimista
4. ✅ **Regimes** - Identificação de ciclos
5. ✅ **Validação** - Métricas de erro
6. ✅ **Visualizações** - Gráficos profissionais

### Próximos Passos:

- Integrar com modelos DSGE
- Usar em análise de viabilidade
- Alimentar sistemas de Reinforcement Learning
- Dashboard interativo